In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
df = pd.read_csv('..\data\dataset_new_score_v3.csv', index_col=0)
df.head()

,titre,budget,recette,duree,realisateur,casting,compagnies_production,suite,Comedy,Drama,...,director_score_sum,director_score_mean,director_score_med,director_score_min,director_score_max,compagnies_score_sum,compagnies_score_mean,compagnies_score_med,compagnies_score_min,compagnies_score_max
0,Dumb and Dumber To,40000000,169837010,110,"['Bobby Farrelly', 'Peter Farrelly']","['Jim Carrey', 'Jeff Daniels', 'Laurie Holden'...","['New Line Cinema', 'Universal Pictures', 'Con...",3,1,0,...,27,13.5,13,13,14,135,33.75,18,0,99
1,Марс,2000000,240000,100,['Anna Melikyan'],"['Nana Kiknadze', 'Artur Smolyaninov', 'Yevgen...","['Central Partnership', 'Studio SLON']",1,1,1,...,0,0.0,0,0,0,0,0.00,0,0,0
2,Captain America: The Winter Soldier,170000000,714766572,136,"['Anthony Russo', 'Joe Russo']","['Chris Evans', 'Scarlett Johansson', 'Sebasti...",['Marvel Studios'],2,0,0,...,128,64.0,64,64,64,81,81.00,81,81,81
3,Тот ещё Карлосон!,4000000,9938268,80,['Sarik Andreasyan'],"['Mikhail Galustyan', 'Igor Vernik', 'Mariya S...",['Enjoy Movies'],1,1,0,...,0,0.0,0,0,0,0,0.00,0,0,0
4,The Saint,68000000,118063304,116,['Phillip Noyce'],"['Val Kilmer', 'Elisabeth Shue', 'Rade Šerbedž...","['Paramount', 'Mace Neufeld Productions', 'Rys...",1,0,0,...,11,11.0,11,11,11,9,3.00,0,0,9


In [3]:
df['compagnies_score_mean'] = df['compagnies_score_mean'].map(int)
df['casting_score_mean'] = df['casting_score_mean'].map(int)
df['director_score_mean'] = df['director_score_mean'].map(int)
df['benefice global'] = df['benefice global'].map(int)
df.drop(columns = ['titre', 'realisateur', 'casting', 'compagnies_production', 'recette'], inplace = True)

In [43]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from random import shuffle
from sklearn.metrics import precision_score, recall_score, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

In [5]:
df['tranche benef'] = 0
df.loc[df['benefice global'] >= df['benefice global'].quantile(0.25), 'tranche benef'] = 1
df.loc[df['benefice global'] >= df['benefice global'].quantile(0.5), 'tranche benef'] = 2
df.loc[df['benefice global'] >= df['benefice global'].quantile(0.75), 'tranche benef'] = 3

In [6]:
X = df.drop(columns = ['tranche benef', 'benefice global'])
Y = df['tranche benef']

In [9]:
data_dmatrix = xgb.DMatrix(data=X,label=Y)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

In [26]:
# Instatiate a XGBClassifier
params = { "objective": "multi:softmax", 'num_class': 4}
xgb_clf = xgb.XGBRegressor(**params)

## Set max_depth to be 10 instead of 100
xgb_clf.set_params(n_estimators=100)

# Fit it to the training set
xgb_clf.fit(X_train, Y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)



In [27]:
## Set max_depth to be 10 instead of 100
xgb_clf.set_params(n_estimators=10)

# Fit it to the training set
xgb_clf.fit(X_train, Y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

In [28]:
print(classification_report(Y_test, preds))

              precision    recall  f1-score   support

           0       0.42      0.55      0.48       237
           1       0.28      0.24      0.26       246
           2       0.33      0.28      0.30       237
           3       0.39      0.39      0.39       219

    accuracy                           0.36       939
   macro avg       0.36      0.36      0.36       939
weighted avg       0.35      0.36      0.36       939



In [14]:
df['tranche benef binaire'] = 0
df.loc[df['benefice global'] > 0, 'tranche benef binaire'] = 1
df['tranche benef binaire'].value_counts()

1    3122
0    1573
Name: tranche benef binaire, dtype: int64

In [30]:
X_bin = df.drop(columns = ['tranche benef', 'benefice global', 'tranche benef binaire'])
Y_bin = df['tranche benef binaire']

In [33]:
X_train_bin, X_test_bin, Y_train_bin, Y_test_bin = train_test_split(X_bin, Y_bin, test_size=0.2, random_state=123)

In [36]:
# Instatiate a XGBClassifier
params = { "objective": "multi:softmax", 'num_class': 2}
xgb_clf = xgb.XGBRegressor(**params)

## Set max_depth to be 10 instead of 100
xgb_clf.set_params(n_estimators=100)

# Fit it to the training set
xgb_clf.fit(X_train_bin, Y_train_bin)

# Predict the labels of the test set
preds_bin = xgb_clf.predict(X_test_bin)

In [37]:
print(classification_report(Y_test_bin, preds_bin))

              precision    recall  f1-score   support

           0       0.50      0.33      0.40       331
           1       0.69      0.82      0.75       608

    accuracy                           0.65       939
   macro avg       0.60      0.58      0.58       939
weighted avg       0.63      0.65      0.63       939



In [44]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [46]:
n_estimators = [100]
max_features = ['auto']
max_depth = [2, 3, 5]
max_depth.append(None)
min_samples_split = [2, 5]
min_samples_leaf = [1, 2, 4]
bootstrap = [True]

params_grid = {'n_estimators': n_estimators, 'max_features': max_features,
               'max_depth': max_depth, 'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}

rf_clf = RandomForestClassifier(random_state=42)

rf_cv = GridSearchCV(rf_clf, params_grid, scoring="accuracy", cv=3, verbose=2, n_jobs=-1)


rf_cv.fit(X_train_bin, Y_train_bin)
best_params = rf_cv.best_params_
print(f"Best parameters: {best_params}")

rf_clf = RandomForestClassifier(**best_params)
rf_clf.fit(X_train_bin, Y_train_bin)

print_score(rf_clf, X_train_bin, Y_train_bin, X_test_bin, Y_test_bin, train=True)
print_score(rf_clf, X_train_bin, Y_train_bin, X_test_bin, Y_test_bin, train=False)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


c:\School_projects\DaSci_project\MovieHype\venv\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters: {'bootstrap': True, 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}


c:\School_projects\DaSci_project\MovieHype\venv\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Train Result:
Accuracy Score: 85.01%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.906587     0.833961  0.850106     0.870274      0.857976
recall        0.609501     0.968974  0.850106     0.789237      0.850106
f1-score      0.728936     0.896412  0.850106     0.812674      0.841033
support    1242.000000  2514.000000  0.850106  3756.000000   3756.000000
_______________________________________________
Confusion Matrix: 
 [[ 757  485]
 [  78 2436]]

Test Result:
Accuracy Score: 67.41%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy   macro avg  weighted avg
precision    0.579618    0.693095  0.674121    0.636356      0.653094
recall       0.274924    0.891447  0.674121    0.583186      0.674121
f1-score     0.372951    0.779856  0.674121    0.576403      0.636421
support    331.000000  608.000000  0.674121 